In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "raw" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.head()

In [ ]:
rides_cp = rides.copy()
rides_cp["duration"] = rides["tpep_dropoff_datetime"] - rides["tpep_pickup_datetime"]
rides_cp.head()

In [ ]:
rides_cp["duration"].describe().T

In [ ]:
rides_cp["duration"].quantile(0)
rides_cp["duration"].quantile(0.01)
rides_cp["duration"].quantile(0.995)
rides_cp["duration"].quantile(0.999)

In [ ]:
duration_filter = (rides_cp["duration"] > pd.Timedelta(0)) & (rides_cp["duration"] <= pd.Timedelta(hours=5))
sum(~duration_filter)

In [ ]:
rides_cp["total_amount"].describe().T

In [ ]:
rides_cp["total_amount"].quantile(0.0)
rides_cp["total_amount"].quantile(0.01)
rides_cp["total_amount"].quantile(0.995)
rides_cp["total_amount"].quantile(0.999)

In [ ]:
rides_cp["total_amount"].max()

In [ ]:
total_amount_filter = (rides_cp["total_amount"]  > 0) & (rides_cp["total_amount"] <= rides_cp["total_amount"].quantile(0.999))
sum(~total_amount_filter) / rides_cp.shape[0] * 100

In [ ]:
rides_cp["total_amount"].plot.box(title="Box Plot of Categories", grid=True)

In [ ]:
nyc_locations = ~rides_cp["PULocationID"].isin((1, 264, 265))
sum(~nyc_locations)                                        

In [ ]:
sorted_df = rides_cp.sort_values(by="tpep_pickup_datetime", ascending=True)  

# Get the top 10 (smallest) and bottom 10 (largest) values  
top_10 = sorted_df.head(10)  
bottom_10 = sorted_df.tail(10)  

top_10

bottom_10

In [ ]:
filter_date_range = (rides_cp["tpep_pickup_datetime"] >= "2023-01-01") & (rides_cp["tpep_pickup_datetime"] < "2023-02-01")
sum(~filter_date_range)

In [ ]:
final_filter = duration_filter & total_amount_filter & nyc_locations & filter_date_range
numbers_dropped = final_filter.shape[0] - sum(final_filter) # numbers dropped
numbers_dropped
numbers_dropped/final_filter.shape[0] * 100

In [ ]:
rides = rides[final_filter]
rides = rides[["tpep_pickup_datetime", "PULocationID"]]
rides.rename(columns={
    "tpep_pickup_datetime": "pickup_datetime",
    "PULocationID": "pickup_location_id"
}, inplace=True)
rides.head()
year = 2023
month = 1
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"
rides.to_parquet(path, engine="pyarrow", index=False)

In [ ]:
rides[rides["pickup_location_id"] == 2]